# Convert the coordinates (WGS84 Lat+Long) of geotags in pictures from degrees to decimal

Two options are included:

1. Given a file with the coordinates extracted through a bash script. Each processed picture should have an entry in that file similar to the following example:

```
/home/arielle/Desktop/TopographicAnalysis/damagePhotos/IMG_0659.JPG
    exif:GPSLatitude: 40/1, 28/1, 819/100
    exif:GPSLongitude: 80/1, 1/1, 732/100
```

2. Given a directory (without subdirectories) containing all the pictures which coordinates will be extracted and converted. This requires the installation of the package `exifread`.

---

Júlio Caineta

20 Nov 2017

In [ ]:
from csv import writer
import os
from __future__ import division

This is an auxiliary function taken from another project of mine. It's pretty simple though.

In [ ]:
def dms2dec(d, m, s):
    """Convert coordinates in the format (Degrees, Minutes, Seconds) to
    decimal.
    Parameters
    ----------
    d : number
        Degrees.
    m : number
        Minutes.
    s : number
        Seconds.
    Returns
    -------
    float
        Coordinates in decimal format.
    Notes
    -----
    Assumes that data is signalled.
    The conversion is done by the formula
    .. math:: \operatorname{DEC} = \operatorname{DEG} + \operatorname{MIN}/60
        + \operatorname{SEC}/3600.
    """
    return int(d) + float(m) / 60 + float(s) / 3600

## Option 1: Using the `imageData` file with the extracted coordinates

*Note:* this is assuming the extracted coordinates have a missing sign on the longitude (hence the minus in `lon_w = -dms2dec(d, m, s)`).

In [ ]:
fp = '/Users/mac1/Downloads/imageData.txt'

In [ ]:
with open(fp, 'r') as fh, open('coords.csv', 'w') as out:
    csvout = writer(out)
    try:
        while True:
            img_no = fh.readline().strip()[-7:-4]
            lat_r = fh.readline().strip().split('exif:GPSLatitude: ')[1]
            d, m, s = map(eval, lat_r.split(','))
            lat_w = dms2dec(d, m, s)
            lon_r = fh.readline().strip().split('exif:GPSLongitude: ')[1]
            d, m, s = map(eval, lon_r.split(','))
            lon_w = -dms2dec(d, m, s)
            csvout.writerow([img_no, lat_w, lon_w])
            
    except:
        print 'done'

## Option 2: Reading all the pictures in a given directory

*Note:* requires `exifread`.

In [ ]:
picdir = '/Users/mac1/Box Sync/TopoFinalProj/damagePhotos'

In [ ]:
import exifread

In [ ]:
with open('coords.csv', 'w') as out:
    csvout = writer(out)
    for pic in os.listdir(picdir):
        with open(os.path.join(picdir, pic), 'rb') as pic_h:
            img_no = pic[-7:-4]
            tags = exifread.process_file(pic_h)
            d, m, s = map(lambda x: x.num / x.den, tags['GPS GPSLatitude'].values)
            lat = dms2dec(d, m, s)
            d, m, s = map(lambda x: x.num / x.den, tags['GPS GPSLongitude'].values)
            lon = -dms2dec(d, m, s)
            csvout.writerow([img_no, lat, lon])